In [49]:
from keras.models import model_from_json
from keras.preprocessing import sequence
from lib import words as wd
from lib import embedding as em
from lib import text_extraction as te
import pandas as pd
import json
pd.set_option('expand_frame_repr', True)

In [53]:
class PartsExtraction(object):
    def __init__(self, keras_model, reduced_dict, labels, maxlen = 100):
        self._model = keras_model
        self._reduced_dict = reduced_dict
        self._labels = labels
        self._maxlen = maxlen
        
    @staticmethod
    def load_from_files(keras_model_filename, keras_weights_filename,
                        reduced_dict_filename, labels=['poteri', 'assemblea', 'clausola', 'non_riconducibile']):
        with open(keras_model_filename) as f:
            km = model_from_json(f.read())
        km.load_weights(keras_weights_filename)
        with open(reduced_dict_filename) as f:
            rd = json.load(f)
        return PartsExtraction(km, rd, labels)
    
    def _int_sentences(self, sentences):
        splitted_sentences = wd.tokenize_sentences(sentences, min_words=1)
        permitted_words = self._reduced_dict.keys()
        reduced_sentences = list(em.reduce_dictionary(splitted_sentences, permitted_words, min_words=1))
        return [[self._reduced_dict[w] for w in sent] for sent in reduced_sentences]
    
    def extract_parts(self, sentences):
        int_sentences = self._int_sentences(sentences)
        padded_data = sequence.pad_sequences(int_sentences, self._maxlen, padding="pre", truncating="post", value=0, dtype='uint32')
        predictions = self._model.predict(padded_data)
        return [self._labels[i] for i in predictions.argmax(axis=-1)]

In [54]:
labels = ['poteri', 'assemblea', 'clausola', 'non_riconducibile']

pe = PartsExtraction.load_from_files('models/extraction_model_30.json',
                                     'models/extraction_weights_30.h5',
                                     'first_5000_words_extraction.json')

In [60]:
document = '../extraction/esempi_descrizioni/5115612230001.pdf'#'../extraction/files_to_label/4907913200001.pdf'

txt = te.extract_text(document, do_ocr=False, pages=-1)
sentences = wd.sentences_doc(txt)

In [61]:
predictions = pe.extract_parts(sentences)

In [64]:
for p,s in zip(predictions, sentences):
    print('\nPREDICTION: {}\n'.format(p))
    print(s)
    


PREDICTION: non_riconducibile

Repertorio n 9116 Raccolta n 4684 COSTITUZIONE DI SOCIETA' A RESPONSABILITA' LIMITATA Repubblica Italiana L'anno duemiladiciassette il giorno sei del mese di luglio in Pistoia e nel mio studio in Corso Silvano Fedi numero 55 Avanti a me Dottor Nicola Ottavi, Notaio in Pistoia, iscritto al Ruolo del Collegio dei Distretti Notarili Riuniti di Firenze Pistoia e Prato, senza l'assistenza dei testimoni, è presente il signor: - BONET MASSIMO, nato a Perugia (PG) il 9 settembre 1966, residente in Perugia (PG), via Francesca Giostrelli n 10, codice fiscale: BNT MSM 66P09 G478T Detto comparente, cittadino italiano, della cui identità personale, io Notaio sono certo, conviene e stipula quanto segue: A) Viene costituita una società a responsabilità limitata sotto la denominazione sociale "Bio srl" società unipersonale

PREDICTION: non_riconducibile

 B) La società ha sede nel Comune di Perugia

PREDICTION: clausola

 il socio dichiara che per l'iscrizione della soc

In [62]:
df = pd.DataFrame({'sentence':sentences,'prediction':predictions, 'index':})

In [78]:
pivoted = df.pivot(columns='prediction', values='sentence')

In [79]:
pivoted

prediction,assemblea,clausola,non_riconducibile,poteri
0,None,None,Repertorio n 9116 Raccolta n 4684 COSTITUZIONE...,None
1,None,None,B) La società ha sede nel Comune di Perugia,None
2,None,il socio dichiara che per l'iscrizione della ...,None,None
3,None,None,C) La durata della società è fissata fino al ...,None
4,None,None,"- la consulenza aziendale e gestionale, intes...",None
5,None,None,dell organizzazione aziendale con riferimento...,None
6,None,None,della organizzazione di reti di vendita,None
7,None,None,di valutazioni d azienda,None
8,None,None,"di fusioni ed acquisizioni, di passaggi gener...",None
9,None,None,la consulenza amministrativa e l elaborazione...,None


In [86]:
d = {k:list(filter(None, pivoted[k])) for k in labels if k != 'non_riconducibile'}

In [104]:
for k,v in d.items():
    print('\n\n'+k.upper().center(127,'.')+'\n')
    print('.\n\n'.join([s.strip() for s in v]))



...........................................................ASSEMBLEA...........................................................

ARTICOLO 6) COMPETENZA DELL'ASSEMBLEA 1 L'assemblea dei soci delibera sulle materie riservate alla competenza dei soci dal presente atto costitutivo, nonché sugli argomenti che uno o più amministratori o tanti soci che rappresentino almeno un terzo del capitale sociale sottopongano alla sua approvazione.

5) la decisione di compiere operazioni che comportano una sostanziale modificazione dell'oggetto sociale o una rilevante modificazione dei diritti dei soci.

ARTICOLO 7) CONVOCAZIONE DELL'ASSEMBLEA 4 1 L'assemblea è convocata dall'organo amministrativo mediante avviso per lettera raccomandata, fax o posta elettronica certificata, spedito ai soci e da essi ricevuto almeno 8 (otto) giorni prima del giorno fissato per l'assemblea.

2 Nell'avviso di convocazione può già essere contenuta la data per la riunione di un'assemblea di seconda convocazione, che non c